New notebook to test the difference between the tech rec mongo db and gal alum.
Currently, code working for gal alum but not tech recruiters.
1. Copy progress from mongodb_test
2. Replicate the gal alum process from
* Connection to mongo db
* Pull data into df
* Manipulate to correct format
* Create csv of all companies
* Create csvs of individual companies

3. New notebook section for tech recruiters

In [1]:
import pandas as pd
import mongo
# import est

# Galvanize Alumni CSV build

In [37]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'gal_alum')
cursor = mongo.coll.find({}, {'_id':0})
df = pd.DataFrame(cursor)
df = df.T

In [3]:
df = pd.DataFrame(cursor)

In [38]:
# df = df.T
df.head()

,0,1,2,3,4,5,6,7,8,9,...,457,458,459,460,461,462,463,464,465,466
University of Texas at Austin,People Link 404,People Link 404,People Link 404,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Apple,NaN,NaN,NaN,{'Erin Gong': 'https://www.linkedin.com/in/eri...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Dell Technologies,NaN,NaN,NaN,NaN,{'Daniel Donovan': 'https://www.linkedin.com/i...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
IBM,NaN,NaN,NaN,NaN,NaN,{'James Helfrich': 'https://www.linkedin.com/i...,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
National Instruments,NaN,NaN,NaN,NaN,NaN,NaN,No results,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
index = list()
results = list()

for idx, row in df.iterrows():
    index.append(idx)
    results.append(row[row.notnull()].values[0])

clean_df = pd.DataFrame(results, index=index)
clean_df.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df.head()

,Scrape Results
University of Texas at Austin,People Link 404
Apple,{'Erin Gong': 'https://www.linkedin.com/in/eri...
Dell Technologies,{'Daniel Donovan': 'https://www.linkedin.com/i...
IBM,{'James Helfrich': 'https://www.linkedin.com/i...
National Instruments,No results


# Austin Technical Recruiters CSV build
## Load data from mongo

In [6]:
mongo.connect_mongo()
mongo.connect_coll('gal_part_proj', 'ATX_tech_rec')
cursor = mongo.coll.find({}, {'_id':0})

df_tr = pd.DataFrame(cursor)
df_tr = df_tr.T

## Austin Tech Recruiters pipeline error corrections
My dataframe pipeline is failing for the tech recruiters mongo pull. Successively passing the Alumni pull with the same process. Steps to troubleshoot proceed.

In [36]:
# pipeline for ATX Tech Recruiters
index_tr = list()
results_tr = list()

for idx, row in df_tr.iterrows():
    index_tr.append(idx)
    results_tr.append(row[row.notnull()].values[0])

results_tr.insert(0, 'ignore')
index_tr.insert(0, 'ignore')    
clean_df_tr = pd.DataFrame(results_tr, index=index_tr)
clean_df_tr.drop('ignore', axis=0, inplace=True)
clean_df_tr.rename({0: 'Scrape Results'}, axis=1, inplace=True)
clean_df_tr.head()

,Scrape Results
9Gauge Partners,{'Carrie Guillory': 'https://www.linkedin.com/...
Abbott,{'Brandy Williams Broden': 'https://www.linked...
Aceable,"{'Andrew Bolisay (he, him, his)': 'https://www..."
AdbaKx,Company Page 404
Addison Group,{'Hattie Cox': 'https://www.linkedin.com/in/ha...


# Convert DataFrame to CSVs for each company (observations)

# gDrive interaction testing

In [111]:
import gspread